Hi

In [ ]:
import ionerdss as ion

pdb_id = '8y7s'
save_folder = '~/Documents/8y7s_dir'

pdb_model = ion.PDBModel(pdb_id=pdb_id, save_dir=save_folder)

# set standard_output=True to see the determined interfaces
pdb_model.coarse_grain(distance_cutoff=0.35, 
                       residue_cutoff=3,
                       show_coarse_grained_structure=False, 
                       save_pymol_script=False, 
                       standard_output=False)

In [ ]:
pdb_model.regularize_homologous_chains(dist_threshold_intra=3.5, 
                                       dist_threshold_inter=3.5, 
                                       angle_threshold=25.0, 
                                       show_coarse_grained_structure=False, 
                                       save_pymol_script=False, 
                                       standard_output=True)

In [ ]:
simulation = ion.Simulation(pdb_model, save_folder)

In [ ]:
simulation.generate_nerdss_input()

In [ ]:
simulation.modify_mol_file('A', 
                           {'D': [20.0, 20.0, 20.0], 
                            'Dr': [0.2, 0.2, 0.2]}) # Example modification

simulation.print_mol_parameters('A')

In [ ]:
simulation.modify_inp_file({'nItr': 1000, 'timeStep': 0.5, 'timeWrite': 100, 'trajWrite': 100, 'pdbWrite': 100, 'A': 200, 'A(A1) + A(A1) <-> A(A1!1).A(A1!1)': {'onRate3Dka': 2000}}) # Example modification
simulation.print_inp_file()

In [ ]:
simulation.install_nerdss(nerdss_path="~/Documents/8y7s_dir") # this will update the nerdss folder if it already exists

In [ ]:
simulation.run_new_simulations(sim_indices=[1, 2, 3], 
                              sim_dir="~/Documents/8y7s_dir/nerdss_output", 
                              nerdss_dir="~/Documents/8y7s_dir/NERDSS", 
                              parallel=False)